In [1]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.io as pio
import ta
import pandas as pd
import numpy as np
import yfinance as yf
from pandas import DataFrame
# Set the default template to plotly_dark
pio.templates.default = 'plotly_dark'

In [2]:
class StockData:
    
    def __init__(self, symbol, start_date, end_date) -> None:
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
        self.data = yf.download(symbol, start=start_date, end=end_date)
        
    def get_data(self) -> DataFrame:
        return self.data